In [1]:
from datetime import timedelta, datetime
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
data_path = "../data/1min/"  # files will be read from there
imputed_data_path = "../data/imputed/1min/"  # files will be written there

# create target directory if does not exist
try:
    os.mkdir("../data/imputed")
    os.mkdir(imputed_data_path)
except OSError as error:
    print(error)

stocks = [file.split('/')[-1][:-4] for file in os.listdir(data_path)]  # read stock names

# datetime format used in CSV files. For more info: https://strftime.org/
datetime_format = "%Y-%m-%dT%X+03:00"

[Errno 17] File exists: '../data/imputed'


In [3]:
# read csv of a stock
historical_df = pd.read_csv(data_path + "SISE.csv")

In [4]:
# show some rows
historical_df.tail()

,Unnamed: 0,symbol,signal_time,date_time,close,high,low,open,total_quantity,volume,weighted_average
396099,6099,SISE,1609167360000,2020-12-28T17:56:00+03:00,7.24,7.25,7.23,7.24,1186305.0,8.588728e+06,7.239899
396100,6100,SISE,1609167420000,2020-12-28T17:57:00+03:00,7.25,7.25,7.24,7.24,85510.0,6.195058e+05,7.244835
396101,6101,SISE,1609167480000,2020-12-28T17:58:00+03:00,7.25,7.25,7.24,7.24,150157.0,1.088206e+06,7.247118
396102,6102,SISE,1609167540000,2020-12-28T17:59:00+03:00,7.25,7.25,7.24,7.24,455938.0,3.302788e+06,7.243942
396103,6103,SISE,1609167840000,2020-12-28T18:04:00+03:00,7.25,7.25,7.25,7.25,4958976.0,3.595258e+07,7.250000


In [5]:
# convert into numpy array
historical_array = historical_df.to_numpy()

# check array dimensions
print(historical_array.shape)

(396104, 11)


In [6]:
class StockImputer:
    """
    Imputer class for historical stock data. It reads the first entry from a sorted historical
    prices. It works with 1-min tick data. It imputes values using LOCF approach (last observation
    carried forward).
    
    Inputs
        data : np.ndarray
            Raw data with missing values
        
    Outputs:
        
    """
    def __init__(self, data):
        """ Constructor for StockImputer class.  """
        # set attributes
        self.raw_data = data
        self.imputed_data = []
        self.imputed_data_as_array = None

        # set starting and ending dates
        self.START_DATE = datetime.strptime(data[0][3], "%Y-%m-%dT%X+03:00")
        self.START_TIMESTAMP = data[0][2]
        self.END_DATE = datetime.strptime(data[-1][3], "%Y-%m-%dT%X+03:00")
        self.END_TIMESTAMP = data[-1][2]

        # locals, used in calculations
        self.filler_index = 0
        self.parser_index = 0

        self.filler_row = self.raw_data[0].copy()
        self.parser_row = self.raw_data[0]

    def _add_row(self, row, keep_values=False):
        """ Add a row and update filler row. """
        
        if not keep_values:
            self.filler_row[4:] = row[4:]  # take the values of current row
        
        self.imputed_data.append(self.filler_row.copy())  # add row to the imputed dataset

        self.filler_row[0] += 1  # inc index
        self.filler_row[2] += 60000  # inc timestamp

        # inc time string
        self.filler_row[3] = datetime.fromtimestamp(self.filler_row[2] / 1000).strftime("%Y-%m-%dT%X+03:00")

    def _add_until_row(self, row):
        """ Add all missing rows between filler and loop rows,
        update filler row in each step. """
        while self.filler_row[2] != row[2]:
            self._add_row(row, keep_values=True)  # add rows between
        
        self._add_row(row, keep_values=False)  # add last row too
        
    def impute(self):
        """ Impute missing values in data. """
        for row in tqdm(self.raw_data):
            
            if self.filler_row[2] == row[2]:  # if timestamp of 2 rows match, add to imputed array
                self._add_row(row)
                continue
            
            else:
                if (row[2] - self.filler_row[2]) / 60000 > 30:
                    # day or session changed
                    self.filler_row = row.copy()
                    self._add_row(row)
                    continue
                
                else:
                    # some missing values 
                    self._add_until_row(row)
                    continue
        self.imputed_data_as_array = np.array(self.imputed_data)
        

In [7]:
imputer = StockImputer(historical_array[:500])

print(f"""
Start Date: {imputer.START_DATE.strftime(datetime_format)}
End Date: {imputer.END_DATE.strftime(datetime_format)}
""")


Start Date: 2017-01-02T10:00:00+03:00
End Date: 2017-01-03T16:39:00+03:00



In [8]:
imputer.impute()  # fill missing values

100%|██████████| 500/500 [00:00<00:00, 20775.19it/s]


In [10]:
not_imputed_df = pd.DataFrame(data=imputer.raw_data, columns=[i+"_RAW" for i in historical_df.columns])
imputed_df = pd.DataFrame(data=imputer.imputed_data_as_array, columns=[i+"_IMP" for i in historical_df.columns])

pd.concat([not_imputed_df[:25], imputed_df[:25]], axis = 1)[['date_time_RAW', 'high_RAW', 'date_time_IMP', 'high_IMP']]

,date_time_RAW,high_RAW,date_time_IMP,high_IMP
0,2017-01-02T10:00:00+03:00,3.093954,2017-01-02T10:00:00+03:00,3.093954
1,2017-01-02T10:03:00+03:00,3.085876,2017-01-02T10:01:00+03:00,3.093954
2,2017-01-02T10:04:00+03:00,3.085876,2017-01-02T10:02:00+03:00,3.093954
3,2017-01-02T10:05:00+03:00,3.093954,2017-01-02T10:03:00+03:00,3.085876
4,2017-01-02T10:06:00+03:00,3.077798,2017-01-02T10:04:00+03:00,3.085876
5,2017-01-02T10:07:00+03:00,3.085876,2017-01-02T10:05:00+03:00,3.093954
6,2017-01-02T10:08:00+03:00,3.077798,2017-01-02T10:06:00+03:00,3.077798
7,2017-01-02T10:09:00+03:00,3.077798,2017-01-02T10:07:00+03:00,3.085876
8,2017-01-02T10:10:00+03:00,3.06972,2017-01-02T10:08:00+03:00,3.077798
9,2017-01-02T10:12:00+03:00,3.061641,2017-01-02T10:09:00+03:00,3.077798


In [111]:
imputer_full = StockImputer(historical_array)

print(f"""
Start Date: {imputer_full.START_DATE.strftime(datetime_format)}
End Date: {imputer_full.END_DATE.strftime(datetime_format)}
""")

imputer_full.impute()

  3%|▎         | 11607/423004 [00:00<00:07, 57248.94it/s]


Start Date: 2017-01-02T10:00:00+03:00
End Date: 2020-12-28T18:04:00+03:00



100%|██████████| 423004/423004 [00:05<00:00, 79761.86it/s]


In [112]:
print(imputer_full.raw_data.shape)
print(imputer_full.imputed_data_as_array.shape)
ratio = (imputer_full.imputed_data_as_array.shape[0] / imputer_full.raw_data.shape[0] - 1) * 100
print(f"Missing values: {round(ratio, 2)}%")

(423004, 11)
(442228, 11)
Missing values: 4.54%


# Impute All Stocks

In [114]:
for stock in stocks:
    historical_df = pd.read_csv(data_path + stock + ".csv")
    historical_array = historical_df.to_numpy()
    
    imputer = StockImputer(historical_array)
    imputer.impute()
    
    imputed_df = pd.DataFrame(data=imputer.imputed_data_as_array, columns=historical_df.columns)
    
    imputed_df.to_csv(imputed_data_path + stock + "_full.csv")


100%|██████████| 359190/359190 [00:05<00:00, 61348.04it/s]
